In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/My Drive/Seq2Seq/BERT/Data'
!ls

/content/drive/My Drive/Seq2Seq/BERT/Data
model.h5  __pycache__  test.csv  tokenization.py  train.csv


In [ ]:
import torch

import random
import numpy as np
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import torchtext
#from torchtext.data import Field,BucketIterator,TabularDataset
from torchtext.legacy.data import Field,BucketIterator,TabularDataset

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!pip install transformers

Data Loading:

Dataset containing a set of sentences with labels 0 and 1 
(1 carrying negaitive sense and 0 carrying positive sense)

In [ ]:
# Loading data

train = pd.read_csv("/content/drive/My Drive/Seq2Seq/BERT/Data/train.csv")
test = pd.read_csv("/content/drive/My Drive/Seq2Seq/BERT/Data/test.csv")

In [ ]:
train.head()

,text,target
0,he is being put on a stretcher do not want to ...,pos
1,emergency units simulate a chemical explosion ...,neg
2,us national park services tonto national fores...,pos
3,bestnaijamade yr old pkk suicide bomber who de...,neg
4,russian ushanka winter military fur hat xl wit...,pos


In [ ]:
train.target.unique()

array(['pos', 'neg'], dtype=object)

In [ ]:
train['target'] = train['target'].replace([1],'neg')
train['target'] = train['target'].replace([0],'pos')

In [ ]:
train.head()

,text,target
0,he is being put on a stretcher do not want to ...,pos
1,emergency units simulate a chemical explosion ...,neg
2,us national park services tonto national fores...,pos
3,bestnaijamade yr old pkk suicide bomber who de...,neg
4,russian ushanka winter military fur hat xl wit...,pos


In [ ]:
# replacing contraction words

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",'i\'m':'i am', "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

In [ ]:
# dealing with spl characters
punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';',  '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√','∞','θ','÷','α','•','à','−','β','∅','³','π','‘','₹','´','°','™','√²','—–' ]

In [ ]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

Preprocessing Functions

In [ ]:
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else mapping[t.lower()] if t.lower() in mapping else t for t in text.split(" ")])
    return text

In [ ]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, '')
    
    specials = {'\u200b': ' ', '…': ' ', '\ufeff': ''}  # to be updated again upon checking the coverage
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

In [ ]:
def remove_newlines(sent):
  sent = re.sub(r'\s+', " ", sent )
  return sent

In [ ]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
        #x = re.sub('[0-9]', '#', x)
    return x

Data Cleansing Activities

In [ ]:
train['text'] = train['text'].apply(lambda x: clean_contractions(x, contraction_mapping))
test['text'] = test['text'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [ ]:
train['text'] = train['text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
test['text'] = test['text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [ ]:
train['text'] = train['text'].apply(lambda x: clean_contractions(x, contraction_mapping))
test['text'] = test['text'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [ ]:
train['text'] = train['text'].apply(lambda x : x.lower())
test['text'] = test['text'].apply(lambda x : x.lower())

In [ ]:
train['text'] = train['text'].apply(lambda x : remove_newlines(x))
test['text'] = test['text'].apply(lambda x : remove_newlines(x))

In [ ]:
train['text'] = train['text'].apply(lambda x: clean_numbers(x))
test['text'] = test['text'].apply(lambda x: clean_numbers(x))

In [ ]:
train['text'] = train['text'].apply(lambda x: clean_numbers(x))
test['text'] = test['text'].apply(lambda x: clean_numbers(x))

In [ ]:
train.head()

,text,target
0,he is being put on a stretcher do not want to ...,pos
1,emergency units simulate a chemical explosion ...,neg
2,us national park services tonto national fores...,pos
3,bestnaijamade yr old pkk suicide bomber who de...,neg
4,russian ushanka winter military fur hat xl wit...,pos


Data cleansing

In [ ]:
# Loading pretrained BERT tokenizer

import transformers
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# The tokenizer has a vocab attribute which contains the actual vocabulary

len(tokenizer.vocab)  #size of the vocabulary

30522

In [ ]:
# Special tokens

init_token = tokenizer.cls_token  
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [ ]:
# Indexes of the special tokens:

init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [ ]:
# The pretrained BERT model was trained on sequences with a defined maximum length 
#it does not know how to handle sequences longer than it has been trained on. 
# Getting the maximum length of these input size for the version of the transformer being used:

max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [ ]:
# Funtion to handle tokenisation of sentences:

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
from torchtext.legacy import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [ ]:
# Splitting the cleansed training dataset into train and validation sets

train_data , valid_data = train_test_split(train,test_size = 0.2)

In [ ]:
train_data.head(1)

,text,target
3763,ouvindo peace love amp armageddon,pos


In [ ]:
%pwd

'/content/drive/My Drive/Seq2Seq/BERT/Data'

In [ ]:
train_data.to_csv('train.csv',index = False)
valid_data.to_csv('test.csv',index = False)

In [ ]:
data_fields = [('text',TEXT),('label',LABEL)]

In [ ]:
# TabularDataset(Dataset):Defines a Dataset of columns stored in CSV, TSV, or JSON format.
# Create a TabularDataset given a path, file format, and field list

train_data , valid_data = TabularDataset.splits(path ='/content/drive/My Drive/Seq2Seq/BERT/Data' ,train='train.csv', test ='test.csv', format='csv', fields=data_fields)

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(valid_data)}")

Number of training examples: 3898
Number of testing examples: 976


In [ ]:
print(vars(train_data.examples[11]))

{'text': [10047, 5458, 1997, 2035, 2122, 2035, 3669, 6961, 18900, 3334, 2111, 2027, 2069, 2360, 2023, 2000, 4315, 12502, 2304, 3669, 6961, 18900, 3334, 2027, 2123, 2102, 2079, 2505, 2005, 1005, 2035, 3268, 1005, 1048, 2213, 7011, 2080], 'label': 'pos'}


In [ ]:
# building vocabulary for the data labels
LABEL.build_vocab(train_data)

In [ ]:
from torchtext.legacy.data import BucketIterator,TabularDataset

BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE, 
    device = device)

Building the Model

In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [ ]:
# Creating an instance of our model

HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [ ]:
# In order to freeze paramers we need to set their requires_grad attribute to False

for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
def count_parameters(model):  # Counting total number of trainable parameters
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [ ]:
# Checking names of the trainable parameters:

for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


Training the Model

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# Placing the model and criterion onto the GPU (if available)

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

Defining functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Training the Model

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
            
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')


Epoch: 01 | Epoch Time: 1m 6s
	Train Loss: 0.404 | Train Acc: 82.61%
Epoch: 02 | Epoch Time: 1m 7s
	Train Loss: 0.373 | Train Acc: 83.66%
Epoch: 03 | Epoch Time: 1m 7s
	Train Loss: 0.353 | Train Acc: 85.23%
Epoch: 04 | Epoch Time: 1m 6s
	Train Loss: 0.342 | Train Acc: 85.37%
Epoch: 05 | Epoch Time: 1m 7s
	Train Loss: 0.310 | Train Acc: 86.87%


Inference

In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, tokenizer, "This film is great")

0.1056985929608345